# ecFlow eLearning Jupyter Notebook

![ecflow](anim/ecflow_el.gif)

- This is a snapshot of ecflow_ui, the ecFlow GUI
- The top *node* is the **server node**, called eowyn here
- right below are the **suites** nodes which contain **families** and **tasks**
- the family node is a container for families and task
- the **task** node is the leaf node from which a **job** is created by ecflow and **submitted**, locally or remotely
- an **alias** node can be created from the GUI, to dynamically modify the script and variables from a task, for an independent submission, in case of debugging, fixing, It is attached to the task and can be removed directly. Multiple aliases can run simultaneously when the task is well designed (different work directory).
- you can see we can attach **attributes** to nodes: 
    -**label, meter, event** which may be updated from the job to follow its progress, reporting a string, an integer, or a boolean (set)
    -**date, time, trigger** (black), **complete** (blue)
    -when the task has to run, a job is submitted, which reports its **status** so that task colour changes.
    -**late** attribute is alos visible here.

![workflow](anim/ecflow.gif)

# ecFlow components

  - the **ecflow_server**, 
  - the clients (**ecflow_client**, the command line text client, **ecflow_ui** the GUI, and the **python API** for python clients). 
  - A **template language** is understood by ecFLow to transform the **task template scripts** into **job files**.

- the template language is generic, so that _you_ decide which language the job will be, bash, ksh, python, perl,...
- You start describing the tasks to run, their relations and dependencies, as a **suite** in a **definition file**
- You load the suite into the server
- You provide the script templates that the ecFlow server shall read at the moment the jobs shall be generated
- when the dependencies are fulfilled (Date, Time, Trigger, Complete, Inlimit) the job is created and submitted. At that moment the task is **submitted**. If anything is problematic, the task turns **aborted**
   -for example, the job could not be created, or it could not be submitted, maybe the remote system not responding or it is refusing the submission (queue, account, size of the job) 
- the jobs starts and writes its progress into the output file. it reports to ecflow server using the **ecflow_client** 
  -**ecflow_client --init** reports the job **active**
  -**ecflow_client --complete** reports sucessful completion
  -at anytime if something goes wront the jobs shall send the **ecflow_client --abort** command to report the task **aborted**

![mindmap](anim/ecflowmm.png)

![status flow](anim/ecflow_status.gif)

## Status transition
- Initially, when a suite is loaded for the first time, its status is **unknown**, so that no job is immediately submitted. It gives a chance for the suite designer to check that it is as expected:
  - scripts and headers can be read by the server, 
  - that there is no variable used in the script, yet not defined in the suite, 
  - the structure of the suite reflect expectation. 
  - at that time, nodes can be suspended from the GUI, deleted, their position modified.
- The **begin** command will let the ecFlow server to initialise servers variables, nodes statuses, and even start jobs when the server is in **restarted mode** and the suite node is not **suspended**
- At that moment, if some jobs cannot be created/submitted, few tasks may turn **aborted**
- most shall remain **queued**, as the default status, waiting the dependencies to be reached (Date, Time, Trigger, Complete, Inlimit)
- the suite definition may contain some nodes using the attribute **defstatus**. It defines the default status of a node when it is requeued or when the suite is begun.
- transiently, submitted jobs are visible (when submitted to a queuing system, when it can last few seconds)
- when the job is started and calls **ecflow_client --init** the task turns **active**
- when the job is completing **ecflow_client --complete** is called
- a job may face problem, and report that with **ecflow_client --abort**
- the variables **ECF_JOB_CMD** and **ECF_KILL_CMD** can be defined in the suite, and modified throughout, reflecting the command the ecFlow server shall call to submit or kill the job.

In [1]:
%load_ext autoreload
%autoreload 2
%quickref


IPython -- An enhanced Interactive Python - Quick Reference Card

obj?, obj??      : Get help, or more help for object (also works as
                   ?obj, ??obj).
?foo.*abc*       : List names in 'foo' containing 'abc' in them.
%magic           : Information about IPython's 'magic' % functions.

Magic functions are prefixed by % or %%, and typically take their arguments
without parentheses, quotes or even commas for convenience.  Line magics take a
single % and cell magics are prefixed with two %%.

Example magic function calls:

%alias d ls -F   : 'd' is now an alias for 'ls -F'
alias d ls -F    : Works if 'alias' not a python name
alist = %alias   : Get list of aliases to 'alist'
cd /usr/share    : Obvious. cd -<tab> to choose from visited dirs.
%cd??            : See help AND source for magic %cd
%timeit x=10     : time the 'x=10' statement with high precision.
%%timeit x=2**100
x**100           : time 'x**100' with a setup of 'x=2**100'; setup code is not
                   co

[video](anim/ecflow_3min39.mp4)

## Start the ecFlow server

In [2]:
%%bash 
ecflow_start.sh -p 2500

ping server(eowyn:2500) succeeded in 00:00:00.145654  ~145 milliseconds
server is already started
0 S map      17387  3759  0  80   0 - 10072 ep_pol 22:53 ?        00:00:00 ecflow_server Server statistics
   Version                         Ecflow (debug) version(4.12.0) boost(1.53.0) compiler(gcc 8.2.0) protocol(TEXT_ARCHIVE) Compiled on Dec 31 2018 15:05:35
   Status                          RUNNING
   Host                            eowyn
   Port                            2500
   Up since                        2019-Jan-16 22:53:05
   Job sub' interval               60s
   ECF_HOME                        /home/map/ecflow_server
   ECF_LOG                         /home/map/home/map/ecflow_server/eowyn.2500.ecf.log
   ECF_CHECK                       /home/map/ecflow_server/eowyn.2500.check
   Check pt interval               120s
   Check pt mode                   CHECK_ON_TIME
   Check pt save time alarm        20s
   Number of Suites                7
   Request's per 1,5,15,30,60 min

## Start the GUI ecflow_ui

In [3]:
%%bash
# UNCOMMENT following line to launch the GUI
# ecflow_ui &

## Experience the command line client

In [4]:
%%bash
ecflow_client --port 2500 --host localhost --ping

ping server(localhost:2500) succeeded in 00:00:00.001947  ~1 milliseconds


In [5]:
%%bash
# DISPLAY ALL CLIENT COMMANDS
ecflow_client --version
ecflow_client --help

Ecflow (debug) version(4.12.0) boost(1.53.0) compiler(gcc 8.2.0) protocol(TEXT_ARCHIVE) Compiled on Dec 31 2018 15:05:35

Client/server based work flow package:

Ecflow (debug) version(4.12.0) boost(1.53.0) compiler(gcc 8.2.0) protocol(TEXT_ARCHIVE) Compiled on Dec 31 2018 15:05:35

ecflow_client provides the command line interface, for interacting with the server:
Try:

   ecflow_client --help=all       # List all commands, verbosely
   ecflow_client --help=summary   # One line summary of all commands
   ecflow_client --help=child     # One line summary of child commands
   ecflow_client --help=user      # One line summary of user command
   ecflow_client --help=<cmd>     # Detailed help on each command

Commands:

   abort                alter                begin                ch_add               ch_auto_add          
   ch_drop              ch_drop_user         ch_register          ch_rem               ch_suites            
   check                checkJobGenOnly      check_pt   

## Python for suites design, client-server communication, or a script.py as a task template...

In [6]:
%%bash
p=/lib/python2.7/site-packages/ecflow
p=/lib/python3.5/site-packages/ecflow
export PYTHONPATH=$PYTHONPATH:/usr/local/apps/ecflow/current$p:/usr/local$p

In [7]:
import sys
p="/lib/python3.5/site-packages/ecflow"
sys.path.append("/usr/local" + p)
sys.path.append("/usr/local/apps/ecflow/current" + p)
import ecflow
# UNCOMMENT FOLLOWING LINE TO GET FULL man pages
# help(ecflow)

## Define the simplest suite and print it as "text definition file"

In [8]:
%run src/el11_test_suite.py

Creating suite definition
# 4.12.0
suite elearning
  defstatus suspended
  edit ECF_HOME '/home/map/ecflow_server'
  edit ECF_INCLUDE '/home/map/ecflow_server/include'
  edit ECF_FILES '/home/map/ecflow_server/files'
  task t1
endsuite
# enddef

Saving definition to file '/home/map/ecflow_serverelearning.def'


In [9]:
%pycat src/el11_test_suite.py

#!/usr/bin/env python
""" a first suite """
# from __future__ import print_function
import os, sys
if 0: # try:
    from ecf import (Defs, Suite, Defstatus, Edit, Task)
else: # except ImportError, err:
    v = "2.7"; v = "3.5"
    INST = "/usr/local/apps/ecflow/lib/python%s/site-packages/ecflow:" % v
    sys.path.append(INST)
    INST = "/usr/local/lib/python%s/site-packages/ecflow" % v
    sys.path.append(INST)
    from ecf import (Defs, Suite, Defstatus, Edit, Task)
    if 0: raise Exception(#ERR: could not import ecf. Does the following line help?"+
        "\nexport PYTHONPATH=$PYTHONPATH:%s" % INST)

print("Creating suite definition")
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
NAME = os.getenv("SUITE", "elearning")
DEFS = Defs()
DEFS.add(
    Suite(NAME).add(  # simplest suite definition
        Defstatus("suspended"),  # so that jobs do not start immediately
        Edit(ECF_HOME=ECF_HOME,  # where to find jobs + output files
             ECF_FILES=ECF_HOME + "/f

## Create the head.h and tail.h files for all tasks in ECF_INCLUDE directory

In [10]:
%run src/el12_test_suite_include.py

head.h/tail.h files are now created in ECF_INCLUDE /home/map/ecflow_server/include


In [11]:
%pycat src/el12_test_suite_include.py

#!/usr/bin/env python
from __future__ import print_function
import os
HEAD = """#!/bin/bash
set -e # stop the shell on first error
set -u # fail when using an undefined variable
set -x # echo script lines as they are executed

# Defines the variables that are needed for any communication with ECF
export ECF_PORT=%ECF_PORT%    # The server port number
export ECF_HOST=%ECF_HOST%    # The host name where the server is running
export ECF_NAME=%ECF_NAME%    # The name of this current task
export ECF_PASS=%ECF_PASS%    # A unique password
export ECF_TRYNO=%ECF_TRYNO%  # Current try number of the task
export ECF_RID=$$             # record the process id.
                              # Also used for zombie detection
# Define the path where to find ecflow_client
# make sure client and server use the *same* version.
# Important when there are multiple versions of ecFlow
export PATH=/usr/local/apps/ecflow/%ECF_VERSION%/bin:$PATH

# Tell ecFlow we have started
ecflow_client --init=$$

# Define a

## Create the simplest task template in ECF_FILES directory

In [12]:
%run src/el13_test_suite_wrapper.py

The script template file is now created as /home/map/ecflow_server/files/t1.ecf


In [13]:
%pycat src/el13_test_suite_wrapper.py

#!/usr/bin/env python
from __future__ import print_function
import os
ECF_HOME = os.getenv("HOME") + "/ecflow_server"  # from ecflow_start.sh
ECF_FILES = ECF_HOME + "/files"
# script template file as a string:
SCRIPT_TEMPLATE = """#!/bin/bash
%include <head.h>
echo "I am part of a suite that lives in %ECF_HOME%"
%include <tail.h>
"""

if __name__ == '__main__':
    if not os.path.exists(ECF_FILES):
        os.makedirs(ECF_FILES)  # create script template files directory
    NAME = ECF_FILES + "/t1.ecf"
    with open(NAME, "w") as t1ecf:
        print(SCRIPT_TEMPLATE, file=t1ecf)
    print("The script template file is now created as", NAME)


## load or replace the node into the Server with a standalone Python ecFlow client

You may ask: why do we need to write an intermediate text file, which is later loaded with another python script which has to read this file?

We don't need: this is a tutorial which explains step by step. In practice the suite is most often directly loaded from creation as ecflow.Defs with ecflow.Client into the server.

Yet, the text (expanded) suite definition can be read, modified, used with ecflow_client command line, to create, or update a node in an existing suite.

In [15]:
%run src/el14_test_suite_client.py

Checking job creation: .ecf -> .job0



In [16]:
%pycat src/el14_test_suite_client.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecflow
# When no arguments is specified, Client uses bash variables ECF_HOST, ECF_PORT
HOST = os.getenv("ECF_HOST", "localhost")
PORT = int(os.getenv("ECF_PORT", "%d" % (1500 + os.getuid())))
NAME = os.getenv("SUITE", "elearning")
# ecflow_start.sh gives port number 1500+uid:
CLIENT = ecflow.Client(HOST + ":%d" % PORT)
# multiple ways to create a client:
# python -c "import ecflow; help(ecflow.Client)"
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server") + "/"
try:
    CLIENT.ping()
except RuntimeError as err:
    print("#ERR: ping failed: " + str(err))
try:  # read definition from disk and load into the server:
    CLIENT.load("%s.def" % ECF_HOME + NAME)
except RuntimeError as err:
    CLIENT.replace("/%s" % NAME, ECF_HOME + "%s.def" % NAME)
DEBUG = True  # DEBUG = False
if DEBUG:
    print("Checking job creation: .ecf -> .job0")
    print(ecflow.Defs(ECF_HOME + "%s.def" % NAME).check_job_creation())


check_job_creation can run from the client side to identify if jobs have a chance to be created by ecFlow server. You can fix:
- missing task template files
- missing include files
- ecFlow variables used in a script template, yet undefined in the py-def
- missing directories (ECF_HOME, ECF_INCLUDE, ECF_FILES)
- unexpected directories right restriction, read only, or directory not accessible

## Standalone Python client to download and display live server content

In [18]:
%run src/el15_checking_the_result.py

uncomment to print defs


## Delete-Load-Begin a suite... or just replace a node

In [19]:
%run src/el16_client_load.py

Server was restarted
Suite elearning is now begun


In [20]:
%pycat src/el16_client_load.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecflow
# When no arguments specified uses ECF_HOST and/or ECF_PORT,
# Explicitly set host and port using the same client
# For alternative argument list see ecflow.Client.set_host_port()
HOST = os.getenv("ECF_HOST", "localhost")
PORT = int(os.getenv("ECF_PORT", "%d" % (1500 + os.getuid())))
CLIENT = ecflow.Client(HOST + ":%d" % PORT)
try:
    CLIENT.ping()
except RuntimeError as err:
    print("ping failed: " + str(err))
try:
    CLIENT.restart_server()
    print("Server was restarted")
except RuntimeError as err:
    print("Server could not be restarted")
try:
    NAME = os.getenv("SUITE", "elearning")
    CLIENT.begin_suite("/%s" % NAME)
    print("Suite %s is now begun" % NAME)
except RuntimeError as err:
    print("suite %s could not begin" % NAME)


- **restart_server** is issued from the Python client, or from the GUI, **once**, so that jobs can be submitted.
- **begin_suite** must be issued each time the suite is loaded. To prevent that, **replace** is sometimes preferred. load would not overwrite a suite already existing on the server and would report an error. Sometimes, **delete** is called on the live suite, to clear the path to the incoming **load**.

# Going Further

## Add a task

In [21]:
%run src/el21_add_another_task.py

Creating suite definition
replaced node /elearning/t2 into localhost 2500


In [22]:
%pycat src/el21_add_another_task.py

#!/usr/bin/env python
from __future__ import print_function
""" add another task, another manual """
import os
from ecf import (Defs, Defstatus, Suite, Variable, Task, Client)
print("Creating suite definition")
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
NAME = os.getenv("SUITE", "elearning")
DEFS = Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variable("ECF_HOME", ECF_HOME),
        Task("t1"),  # first task
        Task("t2"),  # second task
    ))
SCRIPT_TEMPLATE = """%manual
  Manual for task t2
  Operations: if this task fails, set it to complete, report next working day
  Analyst:    Check something ...
%end
%include <head.h>
echo "I am part of a suite that lives in %ECF_HOME%"
%include <tail.h>

%manual
  There can be multiple manual pages in the same file.
  When viewed they are simply concatenated.
%end
"""

if __name__ == '__main__':
    # create script template file
 

## Add a family

In [23]:
%run src/el22_add_families.py

Creating suite definition
replaced node /elearning into localhost 2500


In [24]:
%pycat src/el22_add_families.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Variables)
print("Creating suite definition")
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
NAME = os.getenv("SUITE", "elearning")
DEFS = ecflow.Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(ECF_HOME=ECF_HOME,
                  ECF_FILES=ECF_HOME + "/files",
                  ECF_INCLUDE=ECF_HOME + "/include"),
        Family("f1").add(  # hosting family
            Task("t1"),    # a first task
            Task("t2"),    # a second task
        )))

if __name__ == '__main__':
    HOST = os.getenv("ECF_HOST", "localhost")
    PORT = int(os.getenv("ECF_PORT", "%d" % (1500 + os.getuid())))
    CLIENT = ecflow.Client(HOST, PORT)

    NODE = "/" + NAME  # replace suite node: add f1, delete t1 t2
    CLIENT.replace(NODE, DEFS)
    print("

## Add variables

Variables are essential to a suite. They are attached to a node as an **attribute** (keyword edit in text defintion file, the native ecflow API is node.add_variable("NAME", "value") and ecf.py used in this tutorial defines them with node.add(Variables(a_dictionnary)).

When preprocessing the task script to generate the job, ecFlow server replace each occurence of a variable (name surrounded by the % ECF_MICRO character) with its value.

A task may inherit a variable from a parent node, or overwrite the inherited value, defining it again.

In [25]:
%run src/el23_add_variable.py

replaced node /elearning into localhost 2500


In [26]:
%pycat src/el23_add_variable.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Task, Variables)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
ECF_FILES = ECF_HOME + "/files"
NAME = os.getenv("SUITE", "elearning")
DEFS = ecflow.Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(  # we can add multiple variables at once
            ECF_HOME=ECF_HOME,    # where job files are created by ecflow
            ECF_FILES=ECF_FILES,  # where to find script templates .ecf
            ECF_INCLUDE=ECF_HOME + "/include",  # where to find head.h tail.h
            SLEEP=1,  # user variable to be inherited by task/families below
        ),
        Task("t1").add(
            Variables(SLEEP=5),  # overwriting with value 5 for task t1
        ),
        Task("t2").add(
            Variables(SLEEP=7),  # overwriting with value 7 for task t2
      

## Add Trigger

The Trigger attribute will prevent the task to run until its expression is true.

The expression may contain reference to other nodes for their status, it may reference variables, events, meters, limits.

It is possible to inhibit triggers in a suite setting "ecf.USE_TRIGGER=False", when we design a suite, and few trigger expression refer to "missing" nodes.

In [27]:
%run src/el24_add_trigger.py

replaced node /elearning/f1 into localhost 2500


In [28]:
%pycat src/el24_add_trigger.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Variables, Trigger)
TASK3 = Task("t3")  # a python object can be set, and added later to the suite
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
NAME = os.getenv("SUITE", "elearning")
DEFS = ecflow.Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_HOME + "/files",      # where to find script template
            ECF_INCLUDE=ECF_HOME + "/include",  # where to find head.h tail.h
            SLEEP=1, ),
        Family("f1").add(
            Task("t1").add(Variables(SLEEP=5)),
            Task("t2").add(Variables(SLEEP=7),
                           Trigger("t1 eq complete")),
            TASK3,
            Task("t4").add(Trigger(["t1", "t2"]))

## Add Complete

The Complete attribute is the counterpart of trigger. It will set the task complete as soon as the expresssion is true, so that the job may not have to run.

In [29]:
%run src/el26_add_complete.py

replaced node /elearning into localhost 2500


In [30]:
%pycat src/el26_add_complete.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Variables, Trigger, Event,
                 Complete)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
NAME = os.getenv("SUITE", "elearning")
DEFS = ecflow.Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(  # add multiple variables at once:
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_HOME + "/files",     # where to find script template
            ECF_INCLUDE=ECF_HOME + "/include",  # where to find head.h tail.h
            SLEEP=5),
        Family("f1").add(
            Task("t1"),
            Task("t2").add(
                Trigger("t1 eq complete"),
                Event("a"),
                Event("1")),
            Task("t3").add(
                Trigger("t2:a")),
            Task(

## Add Event

The event is created as an attribute attached to a node (Task, Family) and updated (set) 
- by the job calling the "ecflow_client --event" command
- or a user, using the command "ecflow_client --alter change event"

In [31]:
%run src/el27_add_event.py

replaced node /elearning into localhost 2500


In [32]:
%pycat src/el27_add_event.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Edit, Trigger, Event, Defs, Client)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
NAME = os.getenv("SUITE", "elearning")
DEFS = Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Edit(  # add multiple variables at once:
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_HOME + "/files",      # where to find script template
            ECF_INCLUDE=ECF_HOME + "/include",  # where to find head.h tail.h
            SLEEP=5),
        Family("f1").add(
            Task("t1"),
            Task("t2").add(
                Trigger("t1 eq complete"),
                Event("a"),
                Event("1")),
            Task("t3").add(
                Trigger("t2:a")),
            Task("t4").add(
                Tr

## Add Meter

The Meter attribute is attached to a Task (or a Family) so that the job (or client) can update its integer value.

In [33]:
%run src/el28_add_meter.py

<_io.TextIOWrapper name='/home/map/ecflow_server/files/t1.ecf' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/home/map/ecflow_server/files/t2.ecf' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/home/map/ecflow_server/files/t3.ecf' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/home/map/ecflow_server/files/t4.ecf' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/home/map/ecflow_server/files/t5.ecf' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/home/map/ecflow_server/files/t6.ecf' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/home/map/ecflow_server/files/t7.ecf' mode='w' encoding='UTF-8'>
replaced node /elearning/f2 into localhost 2500


## Add Date and Time

The Date attribute holds the job to run until the date is achieved. 

A Time attribute prevents the job to be submitted immediately. It can be one value (ex 10:00), or a range of time (with an interval, ). Multiple Time attributes can be attached to the same node. Be careful, that, until the last occurence is met, the task is immediately requeued, so that there should not be any trigger referring to the task complete...

When Date and Time are associated, Date holds first, then the Task wait for the Time to start.

When a parent node is **suspended**, it is not enough for a date or time condition to fall. Yet the task will not start until the suspended node is **resumed**. Some may say these attribute have memory. the **why** command/Panel will show when the next expected task occurence is. The **requeue** command will 'restore consumed token' when the task was **executed** (forced to run) manually with the GUI.

We can attach these attributes to a dummy task and refer to it with a trigger in many cases. That way the task, without time dependency directly attached, can be requeued, without reactivating the time condition.

In [34]:
%run src/el29_add_time_date.py

replaced node /elearning into localhost 2500


In [35]:
%pycat src/el29_add_time_date.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Edit,
                 Time, Date, Day, Clock, Defs, Client)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
NAME = os.getenv("SUITE", "elearning")
DEFS = Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Clock("real"),
        Defstatus("suspended"),  # so that jobs do not start immediately
        Edit(  # we can add multiple variables at once
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_HOME + "/files",  # where to find script templates
            ECF_INCLUDE=ECF_HOME + "/include",  # where to find head.h tail.h
            SLEEP=2, ),
        Family("f2").add(
            Task("t1").add(Time("00:30 23:30 00:30")),
            Task("t2").add(Day("sunday")),
            Task("t3").add(Date("16.01.2018"),
                           Time("12:00")),
            Task("t

## Add Label

A label is a text message attached to the task (or Family) which is updated by the task (or client).

In [36]:
%run src/el30_add_label.py

replaced node /elearning into localhost 2500


In [37]:
%pycat src/el30_add_label.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Variables, Label, Meter)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
ECF_INCLUDE = ECF_HOME + "/include"
NAME = os.getenv("SUITE", "elearning")
DEFS = ecflow.Defs()
DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(  # we can add multiple variables at once
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_HOME + "/files",  # where to find script template
            ECF_INCLUDE=ECF_INCLUDE,  # where to find head.h tail.h
            SLEEP=2, ),
        Family("f3").add(
            Task("t1").add(
                Label("info", "none"),
                Meter("step", -1, 240)))))

SCRIPT_TEMPLATE = """#!/bin/bash
%include <head.h>
STEP=0
while [[ $STEP -le 240 ]] ; do
  sleep %SLEEP:1%; ecflow_cli

## Add Repeat

A Repeat is like a (for) loop at suite level. It gets incremented to the next value, once all nodes below get complete. It is an **active** attribute in the sense that it causes the nodes below to be requeued (default status, event and meter reset) when the increment occurs.

In [38]:
%run src/el31_add_repeat.py

replaced node /elearning/f5 into localhost 2500


In [39]:
%pycat src/el31_add_repeat.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Variables, Label, Repeat)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
DEFS = ecflow.Defs()
NAME = os.getenv("SUITE", "elearning")


def create_families():
    """ provider """
    return (
        Family("f4").add(
            Variables(SLEEP=2),
            Repeat("NAME", ["a", "b", "c", "d", "e", "f"], kind="enumerated"),
            Task("t1")),
        Family("f5").add(
            Repeat("DATE", 20170101, 20200105, kind="date"),
            Task("t1").add(
                Repeat("PARAM", 1, 10, kind="integer"),
                Label("info", ""))))


DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(  # we can add multiple variables at once
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_H

## Add Limit and Inlimit

A limit may prevent jobs to be submitted immediately. It can represent a mutex (value 1) or a semaphore.

The Inlimit attribute registers to a limit.

In [40]:
%run src/el32_add_limit.py

replaced node /elearning/f5 into localhost 2500


In [41]:
%pycat src/el32_add_limit.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Variables, Limit, Inlimit)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
DEFS = ecflow.Defs()
NAME = os.getenv("SUITE", "elearning")


def create_family_f5():
    """ provider """
    return Family("f5").add(
        Limit("l1", 2),
        Inlimit("l1"),
        Variables(SLEEP=2),
        [Task("t%d" % tid) for tid in range(1, 10)])


DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(  # we can add multiple variables at once
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_HOME + "/files",  # where to find script templates
            ECF_INCLUDE=ECF_HOME + "/include",  # where to find head.h tail.h
            SLEEP=2, ),
        create_family_f5()))
SCRIPT_TEMPLATE = """#!/bin/bash
%include <

## Add Limit Inlimit

In [42]:
%run src/el32_add_limit.py

replaced node /elearning/f5 into localhost 2500


In [43]:
%pycat src/el32_add_limit.py

#!/usr/bin/env python
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defstatus, Suite, Family, Task, Variables, Limit, Inlimit)
ECF_HOME = os.path.join(os.getenv("HOME"), "ecflow_server")
DEFS = ecflow.Defs()
NAME = os.getenv("SUITE", "elearning")


def create_family_f5():
    """ provider """
    return Family("f5").add(
        Limit("l1", 2),
        Inlimit("l1"),
        Variables(SLEEP=2),
        [Task("t%d" % tid) for tid in range(1, 10)])


DEFS.add(  # suite definition
    Suite(NAME).add(
        Defstatus("suspended"),  # so that jobs do not start immediately
        Variables(  # we can add multiple variables at once
            ECF_HOME=ECF_HOME,  # where job files are created by ecflow
            ECF_FILES=ECF_HOME + "/files",  # where to find script templates
            ECF_INCLUDE=ECF_HOME + "/include",  # where to find head.h tail.h
            SLEEP=2, ),
        create_family_f5()))
SCRIPT_TEMPLATE = """#!/bin/bash
%include <

## Add Late

In [45]:
%run src/el33_add_late.py

replaced node /elearning/f5 into localhost 2500


Late is an attribute which may cause a poping window, to catch attention, when a job remains in submit, or active status for too long, or when complete is not reached in time. In order to really catch attention, some might prefer a **watchdog**, a dedicated task, which turns aborted, beyond a given threshold, or quietly becomes complete, thanks to a Time and Complete attribute.

## Debug a task with an Alias

[alias](anim/ecflow_alias.mp4)

# Exercises

## Data Acquisition suite example

In [49]:
%run src/el41_data_acquisition.py

In [50]:
%pycat src/el41_data_acquisition.py

#!/usr/bin/env python
""" data acquisition suite example """
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Date, Day, Defs, Defstatus, Suite, Family, Task,
                 If,  # If attribute in use example
                 Edit, Label, Repeat, Time, Trigger)
HOME = os.getenv("HOME") + "/ecflow_course"; NAME = "data_acquisition"; DEFS = Defs()
DEFS.add(Suite(NAME).add(
    Defstatus("suspended"),  # so that jobs do not start immediately
    Repeat(kind="day", step=1),
    Edit(ECF_HOME=HOME, ECF_INCLUDE=HOME, ECF_FILES=HOME + "/acq", SLEEP=2),
    [Family(city).add(
        Family("archive").add(
            [Family(obs_type).add(
                If(test=city in ("Exeter", "Toulouse", "Offenbach"), then=Time("00:00 23:00 01:00")),
                If(city in ("Washington"), Time("00:00 23:00 03:00")),
                If(city in ("Tokyo"), Time("12:00")),
                If(city in ("Melbourne"), Day("monday")),
                If(city in ("Montre

## Operational suite?

In [51]:
%run src/el41_data_acquisition.py > /dev/null 2>&1

In [52]:
%pycat src/el41_data_acquisition.py

#!/usr/bin/env python
""" data acquisition suite example """
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Date, Day, Defs, Defstatus, Suite, Family, Task,
                 If,  # If attribute in use example
                 Edit, Label, Repeat, Time, Trigger)
HOME = os.getenv("HOME") + "/ecflow_course"; NAME = "data_acquisition"; DEFS = Defs()
DEFS.add(Suite(NAME).add(
    Defstatus("suspended"),  # so that jobs do not start immediately
    Repeat(kind="day", step=1),
    Edit(ECF_HOME=HOME, ECF_INCLUDE=HOME, ECF_FILES=HOME + "/acq", SLEEP=2),
    [Family(city).add(
        Family("archive").add(
            [Family(obs_type).add(
                If(test=city in ("Exeter", "Toulouse", "Offenbach"), then=Time("00:00 23:00 01:00")),
                If(city in ("Washington"), Time("00:00 23:00 03:00")),
                If(city in ("Tokyo"), Time("12:00")),
                If(city in ("Melbourne"), Day("monday")),
                If(city in ("Montre

## Back archiving

In [54]:
%run src/el42_operational_suite_solution.py > /dev/null 2>&1

In [55]:
%pycat src/el42_operational_suite_solution.py

#!/usr/bin/env python
""" operational suite example """
from __future__ import print_function
import os
import ecf as ecflow
from ecf import (Defs, Defstatus, Suite, Family, Task, Variables,
                 Label, Meter, Repeat, Trigger)
HOME = os.getenv("HOME") + "/ecflow_server"
LAST_STEP = {"12": 240,
             "00": 24, }
NAME = "operational_suite"


def cycle_trigger(cyc):
    if cyc == "12":
        return Trigger("./00==complete")
    return None


DEFS = Defs()
DEFS.add(Suite(NAME).add(
    Defstatus("suspended"),  # so that jobs do not start immediately
    Repeat(kind="day", step=1),
    Variables(ECF_HOME=HOME,
              ECF_INCLUDE=HOME + "/include",
              ECF_FILES=HOME + "/files"),
    [Family(str(cycle)).add(
        Variables(CYCLE=cycle,
                  LAST_STEP=LAST_STEP[cycle]),
        cycle_trigger(cycle),

        Family("analysis").add(
            Task("get_observations"),
            Task("run_analysis").add(Trigger(["get_observations", ])),


## All together

In [58]:
%run src/el51_gallery_suite_example.py > /dev/null 2>&1

In [59]:
%pycat src/el51_gallery_suite_example.py

#!/usr/bin/env python
import os
import ecf as ecflow
from ecf import (
    # Autocancel, Client, Inlimit, Limit, Node, Repeat, Today, Cron, Extern
    Defs, Suite, Family, Task, Clock, Complete, Date, Day, Defstatus,
    Event, Meter, Label, Late, Time, Trigger)
ecflow.USE_LATE = True
HOME = os.getenv('HOME') + '/ecflow_server'


class Edit(ecflow.Variables):
    pass  # rename attribute


def create(name=os.getenv("SUITE", "elearning")):
    return Suite(name).add(
        Defstatus("suspended"),  # start immediately or not for this demo
        Clock("real"),
        Edit(ECF_INCLUDE=HOME,  # header files
             ECF_FILES=HOME,  # script template .ecf
             ECF_HOME=HOME),  # job + local output files
        Family("f1").add(
            Task("t1").add(
                Event(1),
                Label("info", ""),
                Meter("step", -1, 100)),
            Task("t2").add(
                Late("-c 01:00"),
                Meter("step", -1, 100),
                E

# Resources

Confluence ecFlow https://software.ecmwf.int/wiki/display/ECFLOW/ecflow+home

ecFlow, one page, https://software.ecmwf.int/wiki/display/ECFLOW/ecFlow@ECMWF
Tutorial, https://software.ecmwf.int/wiki/display/ECFLOW/Tutorial
User Manual, https://software.ecmwf.int/wiki/display/ECFLOW/User+Manual
Cookbook, https://software.ecmwf.int/wiki/display/ECFLOW/Cookbook
Source code, https://software.ecmwf.int/stash/projects/ECFLOW/repos/ecflow/browse

https://software.ecmwf.int/stash/projects/ECFLOW/repos/elearning/browse

# Glossary

https://software.ecmwf.int/wiki/display/ECFLOW/Glossary